<div  style="min-height: 10px; background-color: lightblue; padding: 20px; border: 2px solid blue; border-radius: 10px; width: 900px; margin: 20px auto;">
    
  <center><strong> Housing Price Prediction using classes </strong></center>

</div>

<div  style="min-height: 10px; background-color: lightblue; padding: 20px; border: 2px solid blue; border-radius: 10px; width: 900px; margin: 20px auto;">
    
  <center><strong> Imports </strong></center>

</div>

In [8]:
import pandas as pd
from scipy.stats.mstats import normaltest
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, PowerTransformer, PolynomialFeatures
from scipy.stats import boxcox
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from scipy.special import inv_boxcox
from sklearn.metrics import r2_score
from sklearn.base import BaseEstimator,TransformerMixin

<div  style="min-height: 10px; background-color: lightblue; padding: 20px; border: 2px solid blue; border-radius: 10px; width: 900px; margin: 20px auto;">
    
  <center><strong> Downloading and Reading Data </strong></center>

</div>

In [5]:
pickle_data=pd.read_pickle('/kaggle/input/boston-housing-clean/boston_housing_clean.pickle')
data=pickle_data['dataframe']

<div  style="min-height: 10px; background-color: lightblue; padding: 20px; border: 2px solid blue; border-radius: 10px; width: 900px; margin: 20px auto;">
    
  <center><strong> Data Cleaning and Analysis </strong></center>

</div>

In [6]:
X=data.drop('MEDV',axis=1)
X.head(5)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


In [7]:
y=data['MEDV']
y.head(5)

0    24.0
1    21.6
2    34.7
3    33.4
4    36.2
Name: MEDV, dtype: float64

In [21]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=999)

<div  style="min-height: 10px; background-color: lightblue; padding: 20px; border: 2px solid blue; border-radius: 10px; width: 900px; margin: 20px auto;">
    
  <center><strong> Forming a model pipeline </strong></center>

</div>

In [47]:
class YTransformer(BaseEstimator,TransformerMixin):
    def __init__(self):
        self.power_transformer=PowerTransformer(method='box-cox')
        
    def fit(self,y,X=None):
        if len(y.shape)==1:
            y_reshaped=y.to_numpy().reshape(-1,1)
        else:
            y_reshaped=y
        self.power_transformer.fit(y_reshaped)
        return self

    def transform(self,y):
        if(len(y.shape)==1):
            y_reshaped=y.to_numpy().reshape(-1,1)
        else:
            y_reshaped=y
        return self.power_transformer.transform(y_reshaped)


    def inverse_transform(self,y):
        return self.power_transformer.inverse_transform(y)
        

In [48]:
X_pipeline=Pipeline(
    [
        ('polynomial', PolynomialFeatures(degree=2, include_bias=False)),
        ('standard scaler', StandardScaler())
    ]
)

In [49]:
y_pipeline=YTransformer()

In [50]:
class RegressionPipeline:
    def __init__(self):
        self.x_pipeline=X_pipeline
        self.y_pipeline=y_pipeline
        self.model=LinearRegression()

    def fit(self,X,y):
        X_transformed= self.x_pipeline.fit_transform(X)
        y_transformed=self.y_pipeline.fit_transform(y)
        self.model.fit(X_transformed,y_transformed)
        return self

    def predict(self,X):
        X_transformed=self.x_pipeline.transform(X)
        y_pred_transformed=self.model.predict(X_transformed)
        return self.y_pipeline.inverse_transform(y_pred_transformed)

    def score(self,X,y):
        y_pred=self.predict(X)
        y_true=y.to_numpy().reshape(-1,1) if len(y.shape)==1 else y
        return self.model.score(self.x_pipeline.transform(X),
                               self.y_pipeline.transform(y_true))

In [51]:
mypipe=RegressionPipeline()

In [52]:
mypipe.fit(X_train,y_train)

In [53]:
y_pred=mypipe.predict(X=X_test)

In [55]:
score=mypipe.score(X_test,y_test)

In [56]:
print(score)

0.7913855180048572


<div  style="min-height: 10px; background-color: lightblue; padding: 20px; border: 2px solid blue; border-radius: 10px; width: 900px; margin: 20px auto;">
    
  <center><strong> MyOwn Model Pipeline </strong></center>

</div>

In [59]:
y_transformer=PowerTransformer(method='box-cox')

y_train_bc=y_transformer.fit_transform(y_train.to_numpy().reshape(-1,1))

In [62]:
pf=PolynomialFeatures(degree=2,include_bias=False)
ss=StandardScaler()
model=LinearRegression()

In [63]:
X_transformer=Pipeline(
    [
        ('poly',pf),
        ('scaler',ss),
        ('model',model)
    ]
)

In [64]:
pipe=X_transformer.fit(X_train,y_train_bc)

In [65]:
y_preds=pipe.predict(X_test)

In [66]:
y_preds_bc=y_transformer.inverse_transform(y_preds)

In [67]:
r2_score(y_test,y_preds_bc)

0.8601417016933022